In [1]:
import csv
import pandas as pd

Visualizzazione del dataset

In [2]:
quadro1 = pd.read_csv("Quadro1.csv",delimiter=',',header=None,
                    names=['prov1','prov2','Frecciarossa al giorno'])
quadro2 = pd.read_csv("Quadro2.csv",delimiter=',',header=None,
                    names=['prov1','prov2','Frecciarossa al giorno'])
quadro3 = pd.read_csv("Quadro3.csv",delimiter=',',header=None,
                    names=['prov1','prov2','Frecciarossa al giorno'])
quadro5 = pd.read_csv("Quadro5.csv",delimiter=',',header=None,
                    names=['prov1','prov2','Frecciarossa al giorno'])

In [3]:
quadro1 #visualizzazione quadro1,quadro2,quadro3,quadro5. 

,prov1,prov2,Frecciarossa al giorno
0,Brescia,Torino,0
1,Brescia,Milano,2
2,Brescia,Piacenza,0
3,Brescia,Parma,0
4,Brescia,Reggio Emilia,0
...,...,...,...
73,Perugia,Napoli,0
74,Perugia,Salerno,0
75,Napoli,Salerno,24
76,Roma,Napoli,92


Inserisco info del dataset in liste

In [4]:
quadro1=[]
quadro2=[]
quadro3=[]
quadro5=[]

with open ("Quadro1.csv") as file:     
    reader = csv.reader(file, delimiter=',')
    for line in reader:
        quadro1.append(line)
with open ("Quadro2.csv") as file:
    reader = csv.reader(file, delimiter=',')
    for line in reader:
        quadro2.append(line)
with open ("Quadro3.csv") as file:
    reader = csv.reader(file, delimiter=',')
    for line in reader:
        quadro3.append(line)
with open ("Quadro5.csv") as file:
    reader = csv.reader(file, delimiter=',')
    for line in reader:
        quadro5.append(line)

### dataset comuni

In [5]:
#visualizzazione con pandas
comuni = pd.read_csv("GeolocComuni.csv",delimiter=',')
comuni

,id_istat,comune,lng,lat,regione
0,1001,Agliè,7.768600,45.363433,Piemonte
1,1002,Airasca,7.484431,44.916886,Piemonte
2,1003,Ala di Stura,7.304344,45.315110,Piemonte
3,1004,Albiano d'Ivrea,7.949145,45.433893,Piemonte
4,1006,Almese,7.395182,45.117663,Piemonte
...,...,...,...,...,...
7899,111103,Villaputzu,9.574832,39.440086,Sardegna
7900,111104,Villasalto,9.388774,39.493758,Sardegna
7901,111105,Villasimius,9.520365,39.142384,Sardegna
7902,111106,Villasor,8.940650,39.380689,Sardegna


In [6]:
comuni = []
# 0 id_istat
# 1 nome comune
# 2 long
# 3 lat
# 4 regione
with open ("GeolocComuni.csv") as file:     ############## da completare
    reader = csv.reader(file, delimiter=',')
    for line in reader:
        comuni.append(line)

### converto i numeri nei quadri in interi e sostituisco eventuali spazi vuoti con zeri

In [7]:
for quadro in [quadro1,quadro2,quadro3,quadro5]:
    for riga in quadro:
        if riga[2]=='' or riga[2]==' ':
            riga[2] = int(0)  ## converte i vuoti in zeri
        else:
            riga[2] = int(riga[2])
        if riga[1][0] == ' ': ## toglie gli spazi prima del nome delle città
            riga[1] = riga[1][1:]
        if riga[1] == 'Reggio Emilia': ## cambio nome reggio emilia per compatibilitá
            riga[1] = "Reggio nell'Emilia"
        if riga[0] == 'Reggio Emilia':
            riga[0] = "Reggio nell'Emilia"
            
### Check
for quadro in [quadro1,quadro2,quadro3,quadro5]:
    for riga in quadro:
        if type(riga[0]) != type(''): # controlla se al posto 0 di ogni riga c'è una stringa
            print('Problema: ', riga)
        elif type(riga[1]) != type(''): # controlla se al posto 1 di ogni riga c'è una stringa
            print('Problema: ', riga)
        elif type(riga[2]) != type(int(0)): # controlla se al posto 2 di ogni riga c'è un intero
            print('Problema: ', riga)
        elif riga[1][0] == ' ': # controlla se al posto 1 di ogni riga, la stringa non inizia con uno spazio
            print('Problema spazio', riga)
        elif len(riga) != 3: # controlla che la lunghezza di ogni riga sia 3
            print('Problema Lunghezza : ', riga)
        elif riga[0][0] == ' ': # controlla se al posto 0 di ogni riga, la stringa non inizia con uno spazio
            print('Problema spazio : ', riga)

## Unisco prima tutti i quadri e poi cerco i "doppioni" e li sommo. Inoltre rimuove le righe con zero collegamenti

In [8]:
def somma_su_doppioni(quadri):
    quadri_new = []
    lista_tratte = []
    for riga in quadri:
        if (riga[0]+riga[1] in lista_tratte) or (riga[1]+riga[0] in lista_tratte):
            try: index = lista_tratte.index(riga[0]+riga[1])
            except: index = lista_tratte.index(riga[1]+riga[0])
            quadri_new[index][2] += riga[2]
        else:
            quadri_new.append(riga)
            lista_tratte = [rigaa[0]+rigaa[1] for rigaa in quadri_new]
    
    return quadri_new
quadri = quadro1+quadro2+quadro3+quadro5

# elimino righe con zero collegamenti
indici_da_eliminare = []
for ii, riga in enumerate(quadri):
    if riga[2]==0:
        indici_da_eliminare.append(ii)

for indice in indici_da_eliminare[::-1]: # vado all'indietro altrimenti gli indici si modificano
    quadri.pop(indice)

# sommo sui doppioni
quadri_new = somma_su_doppioni(quadri)

### Innanzitutto controllo che ci siano solo "andate". Poi creo i ritorni e divido tutti i numeri di treni per due, approssimando per difetto quando serve.

In [9]:
## controllo che ogni tratta compaia solo in una direzione, e non anche inversa.
lista_tratte = lista_tratte = [rigaa[0]+rigaa[1] for rigaa in quadri_new]
for riga in quadri_new:
    if riga[0]+riga[1] in lista_tratte:
        if riga[1]+riga[0] in lista_tratte:
            print('problema')
    elif riga[1]+riga[0] in lista_tratte:
        if riga[0]+riga[1] in lista_tratte:
            print('problema')
    else:
        print('problema !')

In [10]:
## aggiungo anche le tratte inverse
quadri_temp = []
for riga in quadri_new:
    quadri_temp.append([riga[1],riga[0],riga[2]])
quadri_new += quadri_temp

for riga in quadri_new:
    riga[2] = riga[2]//2
    
quadri_new = sorted(sorted(quadri_new,key=lambda x: x[1]), key=lambda x: x[0])
quadri_new

[['Ancona', 'Bari', 4],
 ['Ancona', 'Barletta', 3],
 ['Ancona', 'Bologna', 5],
 ['Ancona', 'Brindisi', 2],
 ['Ancona', 'Foggia', 4],
 ['Ancona', 'Forlì', 1],
 ['Ancona', 'Lecce', 2],
 ['Ancona', 'Milano', 5],
 ['Ancona', 'Pesaro', 3],
 ['Ancona', 'Pescara', 5],
 ['Ancona', "Reggio nell'Emilia", 5],
 ['Ancona', 'Riccione', 2],
 ['Ancona', 'Rimini', 5],
 ['Ancona', 'Termoli', 4],
 ['Ancona', 'Torino', 3],
 ['Bari', 'Ancona', 4],
 ['Bari', 'Barletta', 3],
 ['Bari', 'Bologna', 4],
 ['Bari', 'Brindisi', 2],
 ['Bari', 'Foggia', 4],
 ['Bari', 'Lecce', 2],
 ['Bari', 'Milano', 4],
 ['Bari', 'Pesaro', 3],
 ['Bari', 'Pescara', 4],
 ['Bari', "Reggio nell'Emilia", 4],
 ['Bari', 'Riccione', 2],
 ['Bari', 'Rimini', 4],
 ['Bari', 'Termoli', 4],
 ['Bari', 'Torino', 3],
 ['Barletta', 'Ancona', 3],
 ['Barletta', 'Bari', 3],
 ['Barletta', 'Bologna', 3],
 ['Barletta', 'Brindisi', 2],
 ['Barletta', 'Foggia', 3],
 ['Barletta', 'Lecce', 2],
 ['Barletta', 'Milano', 3],
 ['Barletta', 'Pesaro', 3],
 ['Barletta',

### Controllo che tutte le cittá che abbiamo siano riconosciute nel dataset dei comuni con i codici istat

In [11]:
# 0 id_istat
# 1 nome comune
# 2 long
# 3 lat
# 4 regione
for riga in quadri_new:
    if not riga[0] in [comune[1] for comune in comuni]:
        print("problema: cittá source non trovata", riga[0])
    if not riga[1] in [comune[1] for comune in comuni]:
        print("problema: cittá dest non trovata", riga[1])

### converto nomi città in codici istat

In [12]:
def converti_in_codici_coord(quadro, comuni):
    quadro_conv = []
    nomi_comuni = [comune[1] for comune in comuni]
    for riga in quadro:
        index_source = nomi_comuni.index(riga[0])
        index_dest = nomi_comuni.index(riga[1])
        quadro_conv.append([comuni[index_source][0],comuni[index_source][2],comuni[index_source][3],
                            comuni[index_dest][0],comuni[index_dest][2],comuni[index_dest][3],riga[2]])
    return quadro_conv

def converti_in_codici(quadro, comuni):
    quadro_conv = []
    nomi_comuni = [comune[1] for comune in comuni]
    for riga in quadro:
        index_source = nomi_comuni.index(riga[0])
        index_dest = nomi_comuni.index(riga[1])
        quadro_conv.append([comuni[index_source][0],comuni[index_dest][0],riga[2]])
    return quadro_conv

quadro_prov_codici_coord = converti_in_codici_coord(quadri_new, comuni)
quadro_prov_codici = converti_in_codici(quadri_new, comuni)
#quadro_prov_codici
#contiene in sequenza: codice istat source, long source, lat source, codice istat dest, long dest, lat dest, numero treni

In [13]:
len(quadro_prov_codici_coord)

536

Normalizzo al numero di treni totali, per ottenere una visualizzazione completa del grado degli spostamenti tramite ferrovie AV in Italia.

In [14]:
quadri_prov_prob=[]
i=0
for value in quadro_prov_codici_coord:
    i+=value[6]
i
for value in quadro_prov_codici_coord:
    quadri_prov_prob.append([value[0],value[1],value[2],value[3],value[4],value[5],value[6]/i])
#quadri_prov_prob

#contiene in sequenza: codice istat source, long source, lat source, codice istat dest, long dest, lat dest, probabilità

In [15]:
import folium

def create_map(center=[42,13]):
    return folium.Map(location=center, zoom_start=5)

def add_prov(mappa, prov):
    a = folium.Circle(radius=5e3, location=[float(prov[2]), float(prov[1])],
                         ).add_to(mappa)
    return a

def add_link(mappa, prov):
    b = folium.PolyLine(locations=[[float(prov[2]), float(prov[1])],[float(prov[5]), float(prov[4])]],
                        weight=prov[6]*10**2, color='black').add_to(mappa)
    
    
m = create_map()
for prov in quadri_prov_prob:
    add_prov(m, prov)
    add_link(m, prov)
m.save('mappa_italia_treni.html')
m

# Chiara
# Adesso considero di nuovo la lista "quadro_prov_codici_coord". Ciò che vogliamo fare è normalizzare relativamente al numero di treni della singola città, non a quelli di tutta Italia.  

In [16]:
#cerco come prima cosa il fattore di normalizzazione dei treni per singola privincia
sing_norm=[]
treni=0
c=0
for ii,riga in enumerate(quadro_prov_codici_coord):
    treni+=float(riga[6]) # all'inizio sto sull'ii di riferimento
    c+=1
    if riga[0] != quadro_prov_codici_coord[ii-1][0]:
        if ii !=0:
            treni -= float(riga[6])
            #print(ii)
            for i in range(1,c):
                sing_norm.append(treni)
            c=1
            treni=float(riga[6])
    if ii==535:
        for i in range(0,c):
            sing_norm.append(treni)

In [17]:
len(sing_norm)

536

In [18]:
for i in range(0,536):
    print(quadro_prov_codici_coord[i][0],quadro_prov_codici_coord[i][6],sing_norm[i])

42002 4 53.0
42002 3 53.0
42002 5 53.0
42002 2 53.0
42002 4 53.0
42002 1 53.0
42002 2 53.0
42002 5 53.0
42002 3 53.0
42002 5 53.0
42002 5 53.0
42002 2 53.0
42002 5 53.0
42002 4 53.0
42002 3 53.0
72006 4 47.0
72006 3 47.0
72006 4 47.0
72006 2 47.0
72006 4 47.0
72006 2 47.0
72006 4 47.0
72006 3 47.0
72006 4 47.0
72006 4 47.0
72006 2 47.0
72006 4 47.0
72006 4 47.0
72006 3 47.0
110002 3 39.0
110002 3 39.0
110002 3 39.0
110002 2 39.0
110002 3 39.0
110002 2 39.0
110002 3 39.0
110002 3 39.0
110002 3 39.0
110002 3 39.0
110002 2 39.0
110002 3 39.0
110002 3 39.0
110002 3 39.0
16024 1 5.0
16024 1 5.0
16024 1 5.0
16024 1 5.0
16024 1 5.0
37006 5 410.0
37006 4 410.0
37006 3 410.0
37006 1 410.0
37006 1 410.0
37006 2 410.0
37006 2 410.0
37006 18 410.0
37006 51 410.0
37006 4 410.0
37006 1 410.0
37006 2 410.0
37006 60 410.0
37006 1 410.0
37006 45 410.0
37006 18 410.0
37006 1 410.0
37006 1 410.0
37006 4 410.0
37006 5 410.0
37006 1 410.0
37006 44 410.0
37006 2 410.0
37006 5 410.0
37006 67 410.0
37006 4 41

In [19]:
quadro_prov_sing_norm=[] #contiene le tratte normalizzate in base al numero di treni della singola provincia
for jj,value in enumerate(quadro_prov_codici_coord):
    quadro_prov_sing_norm.append([value[0],value[1],value[2],value[3],value[4],value[5],value[6]/sing_norm[jj]])
quadro_prov_sing_norm

[['42002',
  '13.51887537',
  '43.61675973',
  '72006',
  '16.86737016',
  '41.12559562',
  0.07547169811320754],
 ['42002',
  '13.51887537',
  '43.61675973',
  '110002',
  '16.27717589',
  '41.31955658',
  0.05660377358490566],
 ['42002',
  '13.51887537',
  '43.61675973',
  '37006',
  '11.341720800000001',
  '44.494366799999995',
  0.09433962264150944],
 ['42002',
  '13.51887537',
  '43.61675973',
  '74001',
  '17.94602372',
  '40.63848508',
  0.03773584905660377],
 ['42002',
  '13.51887537',
  '43.61675973',
  '71024',
  '15.54305185',
  '41.46226862',
  0.07547169811320754],
 ['42002',
  '13.51887537',
  '43.61675973',
  '40012',
  '12.04068608',
  '44.22268559',
  0.018867924528301886],
 ['42002',
  '13.51887537',
  '43.61675973',
  '75035',
  '18.17190671',
  '40.35354239',
  0.03773584905660377],
 ['42002',
  '13.51887537',
  '43.61675973',
  '15146',
  '9.1903474',
  '45.46679408',
  0.09433962264150944],
 ['42002',
  '13.51887537',
  '43.61675973',
  '41044',
  '12.91345991',
 

In [20]:
with open('province_norm.csv','w') as f:
    for a in quadro_prov_sing_norm:
        f.write(f'{a[0]},{a[3]},{a[6]}\n')

## Sommo sulle regioni prima convertendo i label in regioni, e poi sommando sui doppioni

In [20]:
def somma_su_doppioni_direzionata(quadri):
    quadri_new = []
    lista_tratte = []
    for riga in quadri:
        if (riga[0]+riga[1] in lista_tratte):
            index = lista_tratte.index(riga[0]+riga[1])
            quadri_new[index][2] += riga[2]
        else:
            quadri_new.append(riga)
            lista_tratte = [rigaa[0]+rigaa[1] for rigaa in quadri_new]
    
    return quadri_new


quadri_regioni = []
lista_comuni = [comune[1] for comune in comuni]
for riga in quadri_new:
    index_source = lista_comuni.index(riga[0])
    index_dest = lista_comuni.index(riga[1])
    quadri_regioni.append([comuni[index_source][4],comuni[index_dest][4],riga[2]])

# cerco treni che viaggiano all'interno della stessa regione
indici_da_eliminare = []
for ii, riga in enumerate(quadri_regioni):
    if riga[0]==riga[1]:
        indici_da_eliminare.append(ii)

for indice in indici_da_eliminare[::-1]: # vado all'indietro altrimenti gli indici si modificano
    quadri_regioni.pop(indice)
quadri_regioni = somma_su_doppioni_direzionata(quadri_regioni)

quadri_regioni = sorted(sorted(quadri_regioni,key=lambda x: x[1]), key=lambda x: x[0])

In [21]:
quadri_regioni

[['Abruzzo', 'Emilia-Romagna', 18],
 ['Abruzzo', 'Lombardia', 5],
 ['Abruzzo', 'Marche', 8],
 ['Abruzzo', 'Molise', 4],
 ['Abruzzo', 'Piemonte', 3],
 ['Abruzzo', 'Puglia', 15],
 ['Campania', 'Emilia-Romagna', 95],
 ['Campania', 'Friuli-Venezia Giulia', 2],
 ['Campania', 'Lazio', 69],
 ['Campania', 'Lombardia', 58],
 ['Campania', 'Piemonte', 23],
 ['Campania', 'Toscana', 51],
 ['Campania', 'Veneto', 28],
 ['Emilia-Romagna', 'Abruzzo', 18],
 ['Emilia-Romagna', 'Campania', 95],
 ['Emilia-Romagna', 'Friuli-Venezia Giulia', 4],
 ['Emilia-Romagna', 'Lazio', 113],
 ['Emilia-Romagna', 'Lombardia', 118],
 ['Emilia-Romagna', 'Marche', 33],
 ['Emilia-Romagna', 'Molise', 14],
 ['Emilia-Romagna', 'Piemonte', 27],
 ['Emilia-Romagna', 'Puglia', 57],
 ['Emilia-Romagna', 'Toscana', 100],
 ['Emilia-Romagna', 'Trentino-Alto Adige', 2],
 ['Emilia-Romagna', 'Umbria', 2],
 ['Emilia-Romagna', 'Veneto', 72],
 ['Friuli-Venezia Giulia', 'Campania', 2],
 ['Friuli-Venezia Giulia', 'Emilia-Romagna', 4],
 ['Friuli-

# lombardia veneto 226 treni: molti di più delle altre regioni. Strano ?

# Ora bisogna associare ad ogni regione il suo capoluogo

In [29]:
#mi serve una lista dove ad ogni codice istat c'è una lista di vicini

 

info_neig = []
coord = []
c = 0
for prov in quadro_prov_sing_norm:
    i = quadro_prov_sing_norm[c][0]
    if i == prov[0]:
        coord.append([prov[5],prov[4],prov[6]]) #lat,long e flusso
    else:
        #info.append(coord)
        info_neig.append([quadro_prov_sing_norm[c][1],quadro_prov_sing_norm[c][2],coord])
        #info = []
        coord = []
    #i = prov[0] 
        c = quadro_prov_sing_norm.index(prov)

 

        
#ogni elemento di info_neig è una lista contenente le seguenti informazioni:
# info_neig[i][0] = long provincia di partenza
# info_neig[i][1] = lat provincia di partenza
# info_neig[i][2] = lista di liste contenente long,lat e prob per ogni provincia di arrivo

In [33]:
quadro_prov_sing_norm

[['42002',
  '13.51887537',
  '43.61675973',
  '72006',
  '16.86737016',
  '41.12559562',
  0.07547169811320754],
 ['42002',
  '13.51887537',
  '43.61675973',
  '110002',
  '16.27717589',
  '41.31955658',
  0.05660377358490566],
 ['42002',
  '13.51887537',
  '43.61675973',
  '37006',
  '11.341720800000001',
  '44.494366799999995',
  0.09433962264150944],
 ['42002',
  '13.51887537',
  '43.61675973',
  '74001',
  '17.94602372',
  '40.63848508',
  0.03773584905660377],
 ['42002',
  '13.51887537',
  '43.61675973',
  '71024',
  '15.54305185',
  '41.46226862',
  0.07547169811320754],
 ['42002',
  '13.51887537',
  '43.61675973',
  '40012',
  '12.04068608',
  '44.22268559',
  0.018867924528301886],
 ['42002',
  '13.51887537',
  '43.61675973',
  '75035',
  '18.17190671',
  '40.35354239',
  0.03773584905660377],
 ['42002',
  '13.51887537',
  '43.61675973',
  '15146',
  '9.1903474',
  '45.46679408',
  0.09433962264150944],
 ['42002',
  '13.51887537',
  '43.61675973',
  '41044',
  '12.91345991',
 